In [1]:
import yahoofinancials as yf    
from quantifin.util.markets import Market
from quantifin.equity.valuation import *
from quantifin.equity import Stock
import json

In [2]:
jpm = Stock("JPM")
nyse = Market(jpm.get_stock_exchange())
price_list = nyse.get_historical_price_data("2010-01-01", "2021-01-01", 'monthly')

In [3]:
market_returns = nyse.get_annualised_return(3)
market_returns


0.0744

In [4]:
#Gordon Growth Model
current_dividend = jpm.full_year_dividend()
growth_rate = jpm.growth_rate()
req_rate = CAPM(0.05, market_returns , jpm.beta)
print(f"Current dividend: {current_dividend} growth rate: {growth_rate} req return {req_rate}")
gordon_growth_valuation(current_dividend, req_rate, growth_rate * 0.5)

Current dividend: 3.6 growth rate: 0.074 req return 0.07174743959999999


107.438

In [5]:
#Forward PE
payout_hist = jpm.get_dividend_payout_ratio_history()
payout = jpm.average_dividend_payout_ratio(payout_hist)
growth_rate = jpm.growth_rate() * 0.5
eps = jpm.get_key_statistics_data()[jpm.stock_code]['trailingEps']
print(f"payout {payout} growth {growth_rate} req_rate {req_rate} EPS {eps}")
forward_pe(payout, growth_rate, req_rate, eps)

payout 0.364 growth 0.037 req_rate 0.07174743959999999 EPS 8.88


96.465

In [9]:
#Multi-stage valuation
s = Stock("NKE")
#Market returns & Req rate of returns
market = Market(s.get_stock_exchange())

market_returns = market.get_annualised_return(5)
print(f"MARKET INFO: Index: {s.get_stock_exchange()} Index returns: {market_returns} Beta: {s.beta}")
req_rate = CAPM(0.03, market_returns, s.beta) * 2

current_dividend = s.full_year_dividend()
growth = s.growth_rate()
proj_growth = [(3, growth), (3, growth * 0.8), (None, growth * 0.4)]  #Projecting growth rate
print(f"Using Dividend: {current_dividend}, Req Return: {req_rate} Growth projection {proj_growth}")

value = multistage_growth(current_dividend, req_rate, proj_growth)
print(f"We arrive at a projected value of: {value}")

MARKET INFO: Index: NYSE Index returns: 0.0788 Beta: 0.86078
Using Dividend: 1.01, Req Return: 0.144012128 Growth projection [(3, 0.178), (3, 0.1424), (None, 0.0712)]
We arrive at a projected value of: 22.667


In [27]:
#Multi-stage valuation using eps (stocks with no dividends)
#s = Stock("AMZN")
market = Market(s.get_stock_exchange())
market_return = market.get_annualised_return(10)
print(f'''
        ======MARKET INFO========
        Index: {market.market_code}  10-year-return: {market_return}''')

req_rate = CAPM(0.03, market_return, s.beta) 
current_eps = s.trailing_eps
growth = s.growth_rate()
proj_growth = [(5, growth), (3, growth * 1.2), (3, growth), (None, growth * 0.7)]
print(f'''
        ===================VALUATION INFO==============
        Required Rate of Return(CAPM): {req_rate}  Current EPS: {current_eps} 
        Projected growth rate: {proj_growth}''')
value = multistage_growth(current_eps, req_rate, proj_growth)
print(f"Value: {value}")


        ======MARKET INFO========
        Index: ^IXIC  10-year-return: 0.1635

        ===================VALUATION INFO==============
        Required Rate of Return(CAPM): 0.1812179865  Current EPS: 41.83 
        Projected growth rate: [(5, 0.189), (3, 0.2268), (3, 0.189), (None, 0.1323)]
Value: 1643.678


In [24]:
##Valuation using FCF
s = Stock("LMT")
market = Market(s.get_stock_exchange())
market_return = market.get_annualised_return(10)
print(f'''
        ======MARKET INFO========
        Index: {market.market_code}  10-year-return: {market_return}''')

req_rate = CAPM(0.05, market_return, s.beta) *2 
no_shares = s.get_num_shares_outstanding()
fcf = s.get_fcf_history()
fcf_growth=s.fcf_growth_rate()*0.1
print(f'''
========Stock info===========
Required Rate of Return(CAPM): {req_rate}  FCF: {fcf} 
fcf_growth: {fcf_growth}   No. of Shares: {no_shares}
''')
value = gordon_growth_valuation(fcf['2020'], req_rate, fcf_growth)
print(f"Valuation: ${round(value/no_shares, 3)}")


        ======MARKET INFO========
        Index: ^NYA  10-year-return: 0.0599

========Stock info===========
Required Rate of Return(CAPM): 0.11871668260000001  FCF: {'2020': 6417000000, '2019': 5827000000, '2018': 1860000000, '2017': 5299000000} 
fcf_growth: 0.05280000000000001   No. of Shares: 280102996.36192936

Valuation: $365.902
